<h2>06. Nihai Üretim Hattı (Final Production Pipeline)</h2>

<hr>

<h3>Proje Detayları</h3>
<ul>
    <li><b>Proje:</b> FreshCart Müşteri Kaybı Tahmini (Customer Churn Prediction)</li>
    <li><b>Amaç:</b> Uçtan Uca Veri İşleme ve Model Eğitimi Hattı (End-to-End Data Processing & Model Training Pipeline)</li>
</ul>

<hr>

<h3>Amaç</h3>
<p>
    Bu betik, önceki tüm adımları (Veri Yükleme, Özellik Mühendisliği ve Modelleme) tek, tekrarlanabilir bir <b>üretim hattında</b> (pipeline) birleştirir. Bir üretim eğitimi çalıştırmasını simüle eder:
</p>

<ol>
    <li><b>Ham Veriyi Yükle</b> (Load Raw Data)</li>
    <li><b>Kesme Stratejisi Uygula</b> (Sızıntıyı Önle) (Apply Cutoff Strategy - Prevent Leakage)</li>
    <li><b>Tüm Özellikleri Oluştur</b> (RFM + Davranışsal + Gelişmiş) (Generate All Features)</li>
    <li><b>Nihai Modeli Eğit</b> (Optimize Edilmiş Hiperparametreleri Kullanarak) (Train Final Model)</li>
    <li><b>Dağıtım İçin Yapıtları Dışa Aktar</b> (Model ve Meta Veriler) (Export Artifacts)</li>
</ol>

<hr>

In [ ]:
#Kütüphaneleri import etme
import numpy as np
import pandas as pd
import lightgbm as lgb
import joblib
import json
import sys
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, classification_report, precision_recall_curve
from scipy import stats

# Proje yapılandırması
from config import RAW_DATA_DIR, PROCESSED_DATA_DIR, MODEL_DIR, RANDOM_STATE, CHURN_DEFINITION

# Veri yükleyici
from data.data_loader import InstacartDataLoader

# Özellik mühendisliği sınıfları
sys.path.append('../src')
from features.rfm_features import RFMFeatureEngineer
from features.behavioral_features import BehavioralFeatureEngineer

print("Kütüphaneler başarıyla yüklendi.")

<h4><b>
Adım 1: Ham Veriyi Al
<h4><b>

In [ ]:
def load_raw_data():
        """Ham Instacart veri kümelerini yükler ve önceki/eğitim siparişlerini birleştirir."""
                
        print("Ham Veri Yükleniyor...")
        
        # config.py dosyasındaki yolu kullanarak veri yükleyiciyi başlat
        loader = InstacartDataLoader(RAW_DATA_DIR)
        
        # Tüm veri kümelerini bir sözlüğe (dictionary) yükle
        data = loader.load_all_data()

        # Ana veri çerçevelerini (dataframes) ayır
        orders_df = data['orders']
        products_df = data['products']
        aisles_df = data['aisles']
        departments_df = data['departments']
        
        # Önceki (prior) ve eğitim (train) sipariş ürünlerini tek bir veri çerçevesinde birleştir (concatenate)
        order_products = pd.concat([
            data['order_products_prior'],
            data['order_products_train']
        ], ignore_index=True)

        print(f"Veri Yüklendi. Siparişler: {len(orders_df):,}, Ürünler: {len(products_df):,}")
        
        # Üretim hattı (pipeline) için gerekli tüm veri çerçevelerini döndür
        return orders_df, products_df, order_products, aisles_df, departments_df

<h4><b>
Adım 2: Özellik Mühendisliği Hattı ("Sızıntısız" Mantık)
<h4><b>

In [ ]:
def calculate_trend(series):
    """
    Zaman serisi eğilimini (slope) hesaplar.
    Hesaplama maliyetinden tasarruf etmek için sadece en az 2 veri noktasına sahip seriler için eğimi (slope) hesaplar.
    """
    if len(series) < 2:
        return 0.0
    try:
        # x zamandır (sipariş sırası), y değerdir (sepet büyüklüğü vb.)
        # stats.linregress, doğrusal regresyon parametrelerini döndürür.
        slope, _, _, _, _ = stats.linregress(np.arange(len(series)), series.values)
        return float(slope)
    except:
        return 0.0

In [ ]:
def run_feature_pipeline(orders_df, order_products, products_df, aisles_df, departments_df):
    """
    03_feature_engineering.ipynb ile tamamen aynı mantıkta, veri sızıntısını önleyerek tüm özellikleri üretir.
    """
    print("\nÖzellik Üretim Hattı Başlatılıyor...")
    
    # 1. Geçmiş ve gelecek ayrımı (leakage önleme)
    print("1. Kesme Stratejisi Uygulanıyor (Geçmiş ve Gelecek Ayrımı Yapılıyor)...")
    orders_sorted = orders_df.sort_values(['user_id', 'order_number'])
    last_orders = orders_sorted.groupby('user_id').tail(1)  # Hedef (Target)
    orders_history = orders_sorted.drop(last_orders.index)  # Özellikler Geçmişi (Features History)
    
    # order_products verilerini sadece geçmiş siparişler için filtrele
    op_history = order_products[order_products['order_id'].isin(orders_history['order_id'])]
    
    # 2. Churn etiketi oluştur
    print("2. Churn Etiketleri Oluşturuluyor...")
    labels = last_orders[['user_id', 'days_since_prior_order']].copy()
    # Eğer son siparişten bu yana geçen gün sayısı >= 30 ise 'churn' (kayıp) olarak etiketle
    labels['is_churn'] = (labels['days_since_prior_order'] >= CHURN_DEFINITION).astype(int)
    labels = labels[['user_id', 'is_churn']]

    # 3. RFM Özellikleri
    print("3. RFM Özellikleri Oluşturuluyor...")
    rfm_engineer = RFMFeatureEngineer(orders_history, op_history)
    rfm_features = rfm_engineer.create_features()
        
    # 4. Davranışsal Özellikler
    print("4. Davranışsal Özellikler Oluşturuluyor...")
    order_products_full = (
        op_history
        .merge(products_df, on='product_id', how='left')
        .merge(aisles_df, on='aisle_id', how='left')
        .merge(departments_df, on='department_id', how='left')
    )
    behavioral_engineer = BehavioralFeatureEngineer(orders_history, order_products_full)
    behavioral_features = behavioral_engineer.create_features()
    
    # 5. Zaman Serisi Eğilimleri
    print("5. Zaman Serisi Eğilimleri Hesaplanıyor...")
    # Sepet büyüklüğü trendi
    basket_sizes = op_history.groupby(['user_id', 'order_id']).size().reset_index(name='basket_size')
    basket_sizes = basket_sizes.merge(orders_history[['user_id', 'order_id', 'order_number']], 
                                    on=['user_id', 'order_id'])
    basket_trend = (
        basket_sizes.sort_values(['user_id', 'order_number'])
        .groupby('user_id')['basket_size']
        .apply(calculate_trend)
        .reset_index(name='basket_size_trend')
    )

    # Sipariş sıklığı ve yenilik ivmesi
    order_freq_trend = (
        orders_history.groupby('user_id')['days_since_prior_order']
        .apply(calculate_trend)
        .reset_index(name='order_frequency_trend')
    )
    recency_accel = (
        orders_history.groupby('user_id')['days_since_prior_order']
        .apply(calculate_trend)
        .reset_index(name='recency_acceleration')
    )
    
    # 6. Satın alma hızı (velocity)
    print("6. Hız ve İvme Metrikleri Oluşturuluyor...")
    user_stats = orders_history.groupby('user_id').agg(
        total_orders=('order_id', 'count'),
        total_days=('days_since_prior_order', 'sum')
    ).reset_index()
    user_stats['purchase_velocity'] = user_stats['total_orders'] / (user_stats['total_days'] + 1)

    # 7. Tüm özellikleri birleştir
    print("7. Özellikler Birleştiriliyor...")
    final_features = (
        labels
        .merge(rfm_features, on='user_id', how='left')
        .merge(behavioral_features, on='user_id', how='left')
        .merge(basket_trend, on='user_id', how='left')
        .merge(order_freq_trend, on='user_id', how='left')
        .merge(recency_accel, on='user_id', how='left')
        .merge(user_stats[['user_id', 'purchase_velocity']], on='user_id', how='left')
        .fillna(0)
    )

    print(f"Pipeline Tamamlandı → Dataset Shape: {final_features.shape}")
    return final_features

<h4><b>
Adım 3: Nihai Model Eğitimi (Final Model Training)
<h4><b>

In [ ]:
def train_model(dataset):
    """Optimize edilmiş hiperparametrelerle LightGBM modeli eğitir."""
    print("\nModel Eğitimi Başlatılıyor...")
    feature_cols = [col for col in dataset.columns if col not in ['user_id', 'is_churn']]
    X = dataset[feature_cols]
    y = dataset['is_churn']

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
    )
    print(f"Veri bölündü → X_train: {X_train.shape}, X_test: {X_test.shape}")

    # 04_model_optimization.ipynb'den gelen en iyi parametreler (örnek)
    best_params = {
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.05,
        'num_leaves': 31,
        'max_depth': 6,
        'min_child_samples': 20,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'reg_alpha': 0.1,
        'reg_lambda': 0.1,
        'random_state': RANDOM_STATE,
        'verbose': -1
    }

    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

    model = lgb.train(
        best_params,
        train_data,
        num_boost_round=500,
        valid_sets=[valid_data],
        callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)]
    )

    # Optimal eşik değeri (F1 maksimizasyonu)
    print("Optimal Eşik Değeri Hesaplanıyor...")
    y_pred_prob = model.predict(X_test)
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred_prob)
    f1_scores = 2 * recall * precision / (recall + precision + 1e-10)
    best_threshold = thresholds[np.argmax(f1_scores)]
    print(f"En İyi Eşik Değeri: {best_threshold:.4f}")

    # Eşik değerini kaydet
    MODEL_DIR.mkdir(parents=True, exist_ok=True)
    with open(MODEL_DIR / 'optimal_threshold.json', 'w') as f:
        json.dump({'optimal_threshold': float(best_threshold)}, f)

    return model, X_test, y_test, feature_cols, best_threshold

<h4><b>
Adım 4: Hızlı Doğrulama ve Sağlama Kontrolü (Quick Validation & Sanity Check)
<h4><b>

In [ ]:
def validate_model(model, X_test, y_test, threshold):
    """Model performansını raporlar."""
    print("\nModel Doğrulanıyor...")
    y_pred_prob = model.predict(X_test)
    auc = roc_auc_score(y_test, y_pred_prob)
    y_pred = (y_pred_prob >= threshold).astype(int)
    f1 = f1_score(y_test, y_pred)

    print(f"Final AUC: {auc:.4f}")
    print(f"Final F1 (threshold {threshold:.4f}): {f1:.4f}")
    print("\nClassification Report:\n", classification_report(y_test, y_pred))

<h4><b>
Adım 5: Dağıtım İçin Ürünleri Dışa Aktar (Export Artifacts for Deployment)
<h4><b>

In [ ]:
def save_artifacts(model, feature_cols, final_dataset):
    """Model ve meta verileri üretim için kaydeder."""
    print("\nÜretim Yapıtları Kaydediliyor...")
    MODEL_DIR.mkdir(parents=True, exist_ok=True)
    PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

    joblib.dump(model, MODEL_DIR / 'final_model_optimized.pkl')
    with open(MODEL_DIR / 'feature_names.json', 'w') as f:
        json.dump(feature_cols, f)
    with open(PROCESSED_DATA_DIR / 'model_features.json', 'w') as f:
        json.dump(feature_cols, f)

    final_dataset.to_parquet(PROCESSED_DATA_DIR / 'final_features_advanced.parquet', index=False)

    print("Tüm yapıtlar başarıyla kaydedildi!")

<h4><b>
ANA ÇALIŞTIRMA (MAIN EXECUTION)
<h4><b>

In [ ]:
if __name__ == "__main__":
    # 1. Veri yükle
    orders_df, products_df, order_products, aisles_df, departments_df = load_raw_data()

    # 2. Özellik mühendisliği
    final_dataset = run_feature_pipeline(orders_df, order_products, products_df, aisles_df, departments_df)

    # 3. Model eğitimi
    model, X_test, y_test, feature_cols, best_threshold = train_model(final_dataset)

    # 4. Doğrulama
    validate_model(model, X_test, y_test, best_threshold)

    # 5. Kaydet
    save_artifacts(model, feature_cols, final_dataset)

    print("\nPIPELINE TAMAMLANDI! Model üretime hazır.")

INFO:data.data_loader:📦 Loading Instacart datasets...
INFO:data.data_loader:   Loading orders.csv...
INFO:data.data_loader:   ✅ Loaded orders: (3421083, 7)
INFO:data.data_loader:   Loading order_products__prior.csv...
INFO:data.data_loader:   ✅ Loaded order_products_prior: (32434489, 4)
INFO:data.data_loader:   Loading order_products__train.csv...
INFO:data.data_loader:   ✅ Loaded order_products_train: (1384617, 4)
INFO:data.data_loader:   Loading products.csv...
INFO:data.data_loader:   ✅ Loaded products: (49688, 4)
INFO:data.data_loader:   Loading aisles.csv...
INFO:data.data_loader:   ✅ Loaded aisles: (134, 2)
INFO:data.data_loader:   Loading departments.csv...
INFO:data.data_loader:   ✅ Loaded departments: (21, 2)
INFO:data.data_loader:✅ All datasets loaded successfully!

INFO:data.data_loader:================================================================================
INFO:data.data_loader:DATA SUMMARY
INFO:data.data_loader:=====================================================

✅ Raw Data Loaded.

⚙️ Starting Feature Pipeline...
   1. Applying Cutoff Strategy (Splitting History vs Future)...
   2. Generating Targets...


INFO:features.rfm_features:🔧 Creating RFM features...
INFO:features.rfm_features:   Creating recency features...


   3. Creating RFM features (Raw Metrics Only)...


INFO:features.rfm_features:   Creating frequency features...
INFO:features.rfm_features:   Creating monetary features (using basket size as a proxy)...
INFO:features.rfm_features:✅ Created 14 RFM features
INFO:features.rfm_features:   Features: ['days_since_last_order', 'days_since_first_order', 'customer_age_days', 'avg_days_between_orders', 'total_orders', 'orders_per_day', 'order_regularity', 'std_days_between_orders', 'avg_basket_size', 'total_items_ordered', 'basket_size_std', 'basket_size_cv', 'avg_unique_products_per_order', 'total_unique_products_ordered']
INFO:features.behavioral_features:🧠 Creating behavioral features...
INFO:features.behavioral_features:   Creating time-based features...


   4. Generating Behavioral Features...


d:\egitim_ve_calismalar\Lodos Makine Öğrenmesi Bootcamp 02.11.2025\html\FreshCart-Churn-Prediction\notebooks\../src\features\behavioral_features.py:104: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weekend_orders = orders_df.groupby('user_id').apply(
d:\egitim_ve_calismalar\Lodos Makine Öğrenmesi Bootcamp 02.11.2025\html\FreshCart-Churn-Prediction\notebooks\../src\features\behavioral_features.py:110: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after grou

   5. Deriving Time-Series Trends (This takes time)...
   6. Generating Velocity & Acceleration Metrics...
   7. Merging Feature Sets...

✅ Pipeline Complete. Dataset Shape: (206209, 46)

🚀 Preparing Data for Training...
✅ Data split complete. X_train shape: (164967, 44), X_test shape: (41242, 44)
✅ Loaded Best Hyperparameters from previous step.

🚀 Training Final LightGBM Model...
Training until validation scores don't improve for 50 rounds
[100]	valid_0's auc: 0.764437
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.764437

⚖️ Calculating Optimal Threshold...
🏆 Best Threshold Found: 0.4272
💾 Threshold saved to: d:\egitim_ve_calismalar\Lodos Makine Öğrenmesi Bootcamp 02.11.2025\html\FreshCart-Churn-Prediction\notebooks\..\models\optimal_threshold.json
✅ Model Training Complete.

📊 Validating Model Performance...
Final AUC Score: 0.7644
Final F1 Score : 0.5902 (at threshold 0.4272)

Classification Report:
              precision    recall  f1-score   support

   